In [55]:
import numpy as np
import geatpy as ea
import random

In [56]:
INDS_LEN = 100
NUM_LIMIT = 10
SUM_CONSTRAINT = 100
POP_SIZE = 5000
GEN_NUM = 500

step_size = 1 / (INDS_LEN / NUM_LIMIT)
block = [round(i * step_size, 3) for i in range(1, INDS_LEN // NUM_LIMIT+1)]
RESULT_LIST = np.array(block * NUM_LIMIT)

In [57]:
# def constraint(Vars):
#     w_num = np.zeros((Vars.shape[0], 1))
#     w_sum = np.zeros((Vars.shape[0], 1))
    
#     for i in range(Vars.shape[0]):
#         rows = Vars[i, :]
#         num_rows = np.where(rows != 0, 1, 0)
#         w_num[i] = np.sum(num_rows)
#         w_sum[i] = np.sum(rows)
    
#     # print("====================================CONSTRAINT====================================")
#     # print("------------------------------------Vars------------------------------------")
#     # print(Vars)
#     # print("------------------------------------NUM------------------------------------")
#     # print(np.abs(w_num - NUM_LIMIT))
#     # print("------------------------------------SUM------------------------------------")
#     # print(np.abs(w_sum - SUM_CONSTRAINT))
    
#     return w_num, w_sum


In [58]:
class MyProblem(ea.Problem):
    
    def __init__(self):
        name = 'MyProblem'
        M = 1
        maxormins = [-1]
        Dim = INDS_LEN
        varTypes = [1] * Dim
        lb = [ 0 for _ in range(INDS_LEN)]
        ub = [ 100 for _ in range(INDS_LEN)]
        lbin = [ 1 for _ in range(INDS_LEN)]
        ubin = [ 1 for _ in range(INDS_LEN)]
        
        ea.Problem.__init__(self,
                            name,
                            M,
                            maxormins,
                            Dim,
                            varTypes,
                            lb,
                            ub,
                            lbin,
                            ubin)
        
    def constraint(self, Vars):
        w_num = np.zeros((Vars.shape[0], 1))
        w_sum = np.zeros((Vars.shape[0], 1))
        
        for i in range(Vars.shape[0]):
            rows = Vars[i, :]
            num_rows = np.where(rows != 0, 1, 0)
            w_num[i] = np.sum(num_rows)
            w_sum[i] = np.sum(rows)
        
        print("====================================CONSTRAINT====================================")
        print("------------------------------------Vars------------------------------------")
        print(Vars)
        print("------------------------------------NUM------------------------------------")
        print(np.abs(w_num - NUM_LIMIT))
        
        return w_num, w_sum
    
    def evalVars(self, Vars):
        f = np.ones((Vars.shape[0], 1))
        
        for i in range(Vars.shape[0]):
            rows = Vars[i, :]
            for j in range(len(rows)):
                w = rows[j]
                if w != 0:
                    f[i] *= w * RESULT_LIST[j]
        
        # print(Vars)
        print(f)
        
        # Penalty
        
        CV = np.hstack(
            [np.abs(self.constraint(Vars)[0] - NUM_LIMIT)])
        
        return f, CV

In [59]:
def create_individual():
    individual = [0] * INDS_LEN

    chosen_positions = random.sample(range(INDS_LEN), NUM_LIMIT)

    remaining = SUM_CONSTRAINT
    for i in range(NUM_LIMIT-1):
        individual[chosen_positions[i]] = random.randint(1, remaining - (NUM_LIMIT - (i + 1)))
        remaining -= individual[chosen_positions[i]]
    individual[chosen_positions[-1]] = remaining

    return individual

def init_pop():
    population = []
    for _ in range(POP_SIZE):
        individual = create_individual()
        population.append(individual)
    return np.array(population)

pop = init_pop()
print(pop)

[[ 0  1  0 ...  0  0  3]
 [ 0  0 34 ...  0  6  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 1  0  0 ...  0  0  0]
 [ 0  0  0 ...  6  0  0]
 [ 0  0  0 ...  0  0  0]]


In [60]:
if __name__ == "__main__":
    problem = MyProblem()
    
    algorithm = ea.soea_DE_currentToBest_1_bin_templet(
        problem,
        ea.Population(Encoding='RI', NIND=POP_SIZE, Chrom=pop),
        MAXGEN=GEN_NUM,
        logTras=100
    )
    
    algorithm.mutOper.F = 0.7
    algorithm.recOper.XOVR = 0.7
    
    res = ea.optimize(
        algorithm,
        verbose=True,
        drawing=1,
        outputMsg=True,
        drawLog=True
    )
    
    print(res)

[[4.44181532e+126]
 [1.20442815e+118]
 [9.35899072e+123]
 ...
 [1.06541540e+120]
 [1.11016244e+123]
 [4.67845976e+118]]
====================================CONSTRAINT====================================
------------------------------------Vars------------------------------------
[[39 67  2 ... 78  7 29]
 [18 60 46 ... 30 80 47]
 [88 17  1 ... 56 27 45]
 ...
 [ 8  8 22 ... 60 75 89]
 [39 68 22 ... 67 86 39]
 [ 5 25 63 ... 37  7  1]]
------------------------------------NUM------------------------------------
[[90.]
 [88.]
 [89.]
 ...
 [88.]
 [89.]
 [89.]]
[[1.04391843e+123]
 [1.04193330e+124]
 [8.19430614e+125]
 ...
 [9.19844332e+120]
 [3.05260861e+127]
 [7.65852798e+121]]
====================================CONSTRAINT====================================
------------------------------------Vars------------------------------------
[[ 37  67  10 ... 100   7  29]
 [ 20 100  46 ...  61  33  54]
 [ 88   3  18 ...  56  27  45]
 ...
 [ 40  10   9 ...  95  44  42]
 [ 26  88  56 ...  67  74  44]


Traceback (most recent call last):
  File "c:\Tool\Python3\lib\site-packages\IPython\core\interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jiawe\AppData\Local\Temp\ipykernel_16356\4046568982.py", line 14, in <module>
    res = ea.optimize(
  File "c:\Tool\Python3\lib\site-packages\geatpy\optimize.py", line 104, in optimize
    [optPop, lastPop] = algorithm.run(prophetPop)
  File "c:\Tool\Python3\lib\site-packages\geatpy\algorithms\soeas\DE\DE_currentToBest_1_bin\soea_DE_currentToBest_1_bin_templet.py", line 73, in run
    self.call_aimFunc(experimentPop)  # 计算目标函数值
  File "c:\Tool\Python3\lib\site-packages\geatpy\Algorithm.py", line 209, in call_aimFunc
    self.problem.evaluation(pop)  # 调用问题类的evaluation()
  File "c:\Tool\Python3\lib\site-packages\geatpy\Problem.py", line 218, in evaluation
    return_object = self.evalVars(pop.Phen)
  File "C:\Users\jiawe\AppData\Local\Temp\ipykernel_16356\1424472661.py", line -1, in 